In [1]:
import cobra
from cobra import Reaction, Metabolite, Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter

# Load Model

In [45]:
model = cobra.io.load_json_model('data/SI3_iJN1411final_flux_w_rhamnolipid_biosynthesis.json')

In [46]:
model.objective = model.reactions.BiomassKT2440_WT3.flux_expression
model.reactions.EX_glyc_e_.lower_bound = 0
model.reactions.ATPM.bounds = -1,1

# Biomass optimization for different carbon nutrients

In [4]:
non_carbon_nutrients_EX = ['EX_nh4_e_',      # ammonium
                           'EX_pi_e_',       # phosphate
                           'EX_so4_e_',      # sulfate
                           'EX_fe3_e_',      # fe3+
                           'EX_mn2_e_',      # mn2+
                           'EX_zn2_e_',      # zn2+
                           'EX_cu2_e_',      # cu2+
                           'EX_ca2_e_',      # ca2+
                           'EX_cl_e_',       # cl-
                           'EX_cobalt2_e_',  # cobalt
                           'EX_k_e_',        # k
                           'EX_mobd_e_',     # mo6+
                           'EX_na1_e_',      # na+
                           'EX_ni2_e_',      # ni2+
                           'EX_mg2_e_',      # mg2+
                           'EX_o2_e_']

carbon_nutrients_EX = []
for rea in model.reactions :
    if rea.id[:2] == 'EX' :
        if not(rea.id in non_carbon_nutrients_EX) :
            carbon_nutrients_EX.append(rea.id)

In [57]:
lb = -10

solutions = {nutrient:0 for nutrient in carbon_nutrients_EX}
for nutrient in carbon_nutrients_EX :
    model.reactions.get_by_id(nutrient).lower_bound = lb
    solutions[nutrient] = int(model.slim_optimize()> 1e-3)
    model.reactions.get_by_id(nutrient).lower_bound = 0